In [134]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType, StringType
from pyspark.sql.functions import sum, map_zip_with, round, col, max, min, stddev, count

In [35]:
spark = SparkSession \
    .builder \
    .master("local[1]") \
    .appName("task") \
    .getOrCreate()

In [36]:
schema = StructType()\
    .add("id",IntegerType(),False)\
    .add("region",StringType(),False)\
    .add("tik",StringType(),False)\
    .add("uik",StringType(),False)\
    .add("includedVoters",IntegerType(),False)\
    .add("recievedBallots",IntegerType(),False)\
    .add("earlyBallots",IntegerType(),False)\
    .add("stationBallots",IntegerType(),False)\
    .add("outStationBallots",IntegerType(),False)\
    .add("canceledBallots",IntegerType(),False)\
    .add("portableBallots",IntegerType(),False)\
    .add("stationaryBallots",IntegerType(),False)\
    .add("invalidBallots",IntegerType(),False)\
    .add("validBallots",IntegerType(),False)\
    .add("lostBallots",IntegerType(),False)\
    .add("notRecordedBallots",IntegerType(),False)\
    .add("BaburinSN",IntegerType(),False)\
    .add("GrudininPN",IntegerType(),False)\
    .add("ZhirinovskyVV",IntegerType(),False)\
    .add("PutinVV",IntegerType(),False)\
    .add("SobchakKA",IntegerType(),False)\
    .add("SuraykinMA",IntegerType(),False)\
    .add("TitovBY",IntegerType(),False)\
    .add("YavlinskyGA",IntegerType(),False)

In [37]:
df = spark.read.format('csv') \
    .option("header",True) \
    .schema(schema) \
    .load("result.csv")

In [38]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- region: string (nullable = true)
 |-- tik: string (nullable = true)
 |-- uik: string (nullable = true)
 |-- includedVoters: integer (nullable = true)
 |-- recievedBallots: integer (nullable = true)
 |-- earlyBallots: integer (nullable = true)
 |-- stationBallots: integer (nullable = true)
 |-- outStationBallots: integer (nullable = true)
 |-- canceledBallots: integer (nullable = true)
 |-- portableBallots: integer (nullable = true)
 |-- stationaryBallots: integer (nullable = true)
 |-- invalidBallots: integer (nullable = true)
 |-- validBallots: integer (nullable = true)
 |-- lostBallots: integer (nullable = true)
 |-- notRecordedBallots: integer (nullable = true)
 |-- BaburinSN: integer (nullable = true)
 |-- GrudininPN: integer (nullable = true)
 |-- ZhirinovskyVV: integer (nullable = true)
 |-- PutinVV: integer (nullable = true)
 |-- SobchakKA: integer (nullable = true)
 |-- SuraykinMA: integer (nullable = true)
 |-- TitovBY: integer (nul

### 1. Найти явку в % по регионам

In [75]:
data = df.select('region','validBallots', 'includedVoters', 'uik') \
    .filter('uik=="Сумма"') \
    .groupBy('region') \
    .agg(sum('validBallots').alias('validBallots_sum'),sum('includedVoters').alias('includedVoters_sum'))

In [80]:
result = data.withColumn('percent', round((data['validBallots_sum'] / data['includedVoters_sum']) * 100, 2)) \
    .sort(col('percent').desc())
result.show()


+--------------------+----------------+------------------+-------+
|              region|validBallots_sum|includedVoters_sum|percent|
+--------------------+----------------+------------------+-------+
|Территория за пре...|          468565|            483957|  96.82|
|     Республика Тыва|          162780|            175102|  92.96|
|Кабардино-Балкарс...|          483932|            528431|  91.58|
|Ямало-Ненецкий ав...|          338067|            370823|  91.17|
|Чеченская Республика|          646885|            709635|  91.16|
|Республика Северн...|          459847|            515157|  89.26|
| Республика Дагестан|         1420661|           1631826|  87.06|
|Карачаево-Черкесс...|          253970|            292031|  86.97|
| Кемеровская область|         1655063|           2005066|  82.54|
|Республика Ингушетия|          179031|            219305|  81.64|
|Чукотский автоном...|           27298|             33541|  81.39|
|    Брянская область|          772301|            978509|  78

### 2. Выбрать любимого кандидата и найти тот избирательный участок, на котором он получил наибольший результат (учитывать участки на которых проголосовало больше 300 человек)

In [90]:
df.select('region','tik', 'uik', 'SobchakKA', 'validBallots') \
    .filter('uik!="Сумма"') \
    .filter('validBallots > 300') \
    .withColumn('percent', round((col('SobchakKA') / col('validBallots')) * 100, 2)) \
    .sort(col('percent').desc()) \
    .show()

+--------------------+--------------------+---------+---------+------------+-------+
|              region|                 tik|      uik|SobchakKA|validBallots|percent|
+--------------------+--------------------+---------+---------+------------+-------+
|Территория за пре...|                null|УИК №8239|      293|        1059|  27.67|
|Территория за пре...|                null|УИК №8061|      868|        3603|  24.09|
|Территория за пре...|                null|УИК №8238|      200|         928|  21.55|
|Территория за пре...|                null|УИК №8311|       90|         446|  20.18|
|Территория за пре...|                null|УИК №8168|       80|         403|  19.85|
|город Санкт-Петер...|Территориальная и...|УИК №1644|      222|        1187|   18.7|
|Свердловская область|Екатеринбург, Вер...|УИК №1318|       86|         464|  18.53|
|  Ростовская область|Ростов-на-Дону, О...|УИК №2305|       57|         309|  18.45|
|        город Москва|     район Печатники|УИК №3709|      150|  

### 3. Найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна

In [94]:
data = df.select('region','validBallots', 'includedVoters', 'uik') \
    .filter('uik=="Сумма"') \
    .withColumn('percent', round((col('validBallots')) / col('includedVoters') * 100, 2)) \
    .groupBy('region') \
    .agg(max('percent').alias('percent_max'),min('percent').alias('percent_min')) \
    .withColumn('difference', round(col('percent_max') - col('percent_min'), 2)) \
    .sort(col('difference').desc()) \
    .show()

+--------------------+-----------+-----------+----------+
|              region|percent_max|percent_min|difference|
+--------------------+-----------+-----------+----------+
|Архангельская обл...|      99.28|       49.7|     49.58|
| Сахалинская область|      98.14|      50.27|     47.87|
| Республика Дагестан|       98.1|      50.72|     47.38|
| Республика Калмыкия|      90.94|      47.73|     43.21|
|   Самарская область|       98.2|      56.32|     41.88|
|   Красноярский край|      89.85|      48.52|     41.33|
| Саратовская область|      92.87|      53.05|     39.82|
|Республика Татарс...|      98.36|      58.76|      39.6|
|  Краснодарский край|      94.66|      58.17|     36.49|
|Республика Адыгея...|       97.2|      60.94|     36.26|
|    Липецкая область|      96.23|      60.14|     36.09|
|     Камчатский край|      98.62|      62.81|     35.81|
| Воронежская область|      89.37|      53.84|     35.53|
|Калининградская о...|      86.79|      51.66|     35.13|
|  Ростовская 

### 4. Посчитать дисперсию по явке для каждого региона (учитывать УИК)

In [132]:
data = df.select('region','validBallots', 'includedVoters', 'uik', round(col('validBallots') / col('includedVoters'), 2).alias('percent')) \
    .filter('uik=="Сумма"') \
    .groupBy('region') \
    .agg(stddev(col('percent')).alias('std')) \
    .show()

+--------------------+--------------------+
|              region|                 std|
+--------------------+--------------------+
|Республика Саха (...| 0.07522291801677189|
|Калининградская о...| 0.07755316082290385|
|Новосибирская обл...|0.045825180260173334|
|   город Севастополь|0.015000000000000024|
|Свердловская область| 0.04508414963185734|
|  Республика Хакасия| 0.04778423728943818|
| Магаданская область| 0.07983315935958774|
|Республика Ингушетия| 0.05833333333333332|
|Удмуртская Респуб...| 0.05058393423659384|
|     Камчатский край| 0.11266046756684864|
| Саратовская область|   0.096450415764799|
|Республика Башкор...|  0.0819607333259752|
|  Ростовская область| 0.08018157606419005|
|   Калужская область| 0.07031580312641085|
|Еврейская автоном...|                0.04|
|Карачаево-Черкесс...| 0.07149380138420142|
| Республика Дагестан| 0.07615681346797598|
|Ханты-Мансийский ...| 0.08015137194235934|
|Республика Татарс...| 0.11640075006366844|
|Ямало-Ненецкий ав...|0.04771711

### 5. Для каждого кандидата посчитать таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

In [144]:
for person in ['BaburinSN', 'GrudininPN', 'ZhirinovskyVV', 'PutinVV', 'SobchakKA', 'SuraykinMA', 'TitovBY', 'YavlinskyGA']:   
    df.select('region','tik', 'uik', person, 'validBallots') \
        .filter('uik!="Сумма"') \
        .withColumn('percent', round((col(person) / col('validBallots')) * 100)) \
        .groupBy('percent') \
        .agg(count('uik').alias('uik_count')) \
        .sort(col('percent').desc()) \
        .show()

+-------+---------+
|percent|uik_count|
+-------+---------+
|   44.0|        1|
|   23.0|        1|
|   21.0|        2|
|   20.0|        2|
|   18.0|        2|
|   17.0|        1|
|   15.0|        2|
|   14.0|        2|
|   13.0|        1|
|   12.0|        1|
|   11.0|        6|
|   10.0|        9|
|    9.0|       10|
|    8.0|       21|
|    7.0|       21|
|    6.0|       37|
|    5.0|       86|
|    4.0|      183|
|    3.0|      577|
|    2.0|     3889|
+-------+---------+
only showing top 20 rows

+-------+---------+
|percent|uik_count|
+-------+---------+
|   80.0|        1|
|   78.0|        1|
|   75.0|        1|
|   71.0|        1|
|   70.0|        1|
|   69.0|        2|
|   67.0|        3|
|   61.0|        1|
|   60.0|        1|
|   58.0|        3|
|   57.0|        6|
|   56.0|        4|
|   55.0|        3|
|   54.0|        5|
|   53.0|        3|
|   52.0|        4|
|   51.0|        4|
|   50.0|       12|
|   49.0|        5|
|   48.0|        8|
+-------+---------+
only showing t